In [1]:
import json
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from trl import SFTTrainer

In [5]:
notebook_login()

In [6]:
dataset = load_dataset("TokenBender/code_instructions_122k_alpaca_style", split="train")

Generating train split:   0%|          | 0/121959 [00:00<?, ? examples/s]

In [24]:
# this function is used to output the right formate for each row in the dataset
def create_text_row(instruction, input, output):
    text_row = f"""<s>[INST] {instruction} here are the inputs {input} [/INST] \\n {output} </s>"""
    return text_row

# interate over all the rows formate the dataset and store it in a jsonl file
def process_jsonl_file(output_file_path):
    with open(output_file_path, "w") as output_jsonl_file:
        j = 0
        for item in dataset:
            json_object = {
                "text": create_text_row(item["instruction"], item["input"] ,item["output"]),
                "instruction": item["instruction"],
                "input": item["input"],
                "output": item["output"]
            }
            output_jsonl_file.write('[')
            output_jsonl_file.write(json.dumps(json_object) + "\\n")
            if (j >= 1):
                break
            j = j + 1
            output_jsonl_file.write('],')
        output_jsonl_file.write(']')
# Provide the path where you want to save the formatted dataset
process_jsonl_file("./training_dataset.jsonl")

In [26]:
train_dataset = load_dataset('json', data_files='./training_dataset.jsonl', split="train")

Generating train split: 0 examples [00:00, ? examples/s]

Failed to load JSON from file '/home/catB/kl279585/projects/cea_stage_salome/notebooks/training_dataset.jsonl' with error <class 'pyarrow.lib.ArrowInvalid'>: JSON parse error: Column() changed from object to array in row 0


DatasetGenerationError: An error occurred while generating the dataset